In [1]:
#Importamos las librerias a utilizar.
import numpy as np
import neattext as nt
import joblib as jbl
import os

In [2]:
def resultado_class(X): #función que clasifica los resultados de la predicción
    media = np.mean(X)
    
    if media >=0.10 and media < 0.15:
        return "Es poco probable que el tweet sea de un desastre"
    elif media >=0.15 and media <0.25:
        return "Es bastante probable que el tweet sea de un desastre"
    elif media >= 0.25 and media <0.50:
        return "Es altamente probable que el tweet sea de un desastre"
    elif media >=0.50 and media <=1:
        return "El tweet es de un desastre"
    else: 
        return "El Tweet no es de un desastre"

In [3]:
def limpiar(text): #función para limpair texto

    text = nt.TextFrame(str(text)).remove_stopwords(lang = "en") #quita stopwords
    text = nt.TextFrame(str(text)).remove_emails() #quita e-mails
    text = nt.TextFrame(str(text)).remove_numbers() #quita números
    text = nt.TextFrame(str(text)).remove_phone_numbers()#quita números de teléfonos
    text = nt.TextFrame(str(text)).remove_btc_address()#quita direcciones
    text = nt.TextFrame(str(text)).remove_urls()#quita urls
    text = nt.TextFrame(str(text)).remove_emojis()#quita emoticones
    text = nt.TextFrame(str(text)).remove_html_tags()#quita tags
    text = nt.TextFrame(str(text)).remove_puncts()#remueve signos de puntuación
    text = nt.TextFrame(str(text)).remove_special_characters() #remueve caracteres especiales
    text = nt.TextFrame(str(text)).remove_stopwords(lang = "en") #vuelve a remover posibles stopwords
    text = text.word_tokens()#tokenize el resultado
    return text


In [5]:
count_vec= jbl.load("vectorizer") #cargamos los modelos para la preparación de datos
tfidf_vec= jbl.load("tfdvectorizer")#cargamos los modelos para la preparación de datos
model_naive = jbl.load("naive_bayes") #cargamos el modelo para predecir 
model_logistic = jbl.load("logistic_regresion")#cargamos el modelo para predecir 
model_logistic_2 = jbl.load("logistic_regresion_V")#cargamos el modelo para predecir 

In [6]:
import dash #importamos las librerías necesarias para trabajar el dashboard
from dash import html
from dash import dcc
from dash.dependencies import Input, Output, State

import warnings
warnings.filterwarnings("ignore")

In [35]:
app = dash.Dash()

app.layout = html.Div(style={'background-image': 'url("https://about.twitter.com/content/dam/about-twitter/en/brand-toolkit/brand-hero-desktop.jpg.twimg.1920.jpg")', }, id = 'parent', children = [
    html.H1(id = 'H1', children = 'Predicción de Tweets', style = {'textAlign':'center',\
                                            'marginTop':50,'marginBottom':50, 'font-family':'Courier New', 'font-size':'70px', 'color':'#00acee'}),
        
        html.H2("Modelos",  style ={'font-family':'Courier New', 'color':'#00acee'}),  #para texto
        dcc.Dropdown(id = 'dropdown', #menu de opciones 
        options = [
            {'label':'Naive - Bayes (Más recomendado)', 'value':'bayes'}, #lsita de opciones "label" el valor que se muestra en pantalla
            {'label': 'Regresion Logistica + Vectorizador (recomendado)', 'value':'regresionLV'}, #value es el valor del dropdown
            {'label': 'Regresion Logistica (menos recomendado)', 'value':'regresionL'},
            {'label':'Todos los Modelos', 'value':'models'},
            ],
        value = 'bayes', style = {'font-family':'Courier New'}), #valor inicial 
    
    dcc.Textarea(
        id='text_area',
        value='',
        style={'width': '99.5%', 'height': 300, 'font-family':'Courier New', 'font-size':'20px'},
    ),
    html.Button('Predecir', id='button', n_clicks=0, style ={'font-family':'Courier New', "font-size": "15px", 'width': '10%', 'height': 40, 'background-color':'#3b83bd', 'color':'white', 'border-radius': '4px'}),
    html.Button("Limpiar", id = "button2", n_clicks = 0, style ={'font-family':'Courier New', "font-size": "15px", 'width': '10%', 'height': 40, 'background-color':'#006494', 'color':'white', 'border-radius': '4px'}),
    html.Button("Gráficas", id = "button3", n_clicks = 0, style ={'font-family':'Courier New', "font-size": "15px", 'width': '10%', 'height': 40, 'background-color':'#006494', 'color':'white', 'border-radius': '4px'}),
    html.Div(id = "model_text", style={'whiteSpace': 'pre-line', "font-size": "large", 'font-family':'Courier New'}),
    html.Div(id='resultado', style={'whiteSpace': 'pre-line', "font-size": "large", 'font-family':'Courier New'}),

    html.Div(id="graph-container", children= [


        dcc.Graph(id = 'graph1',
            figure={
                'data': [
                    {'x': ['Prueba 1', 'Prueba 2', 'Prueba 3', 'Prueba 4'], 'y': [0.7267, 0.8322, 0.8578, 0.8775], 'type': 'bar', 'name': 'Accuracy'},
                ],
                'layout': {
                    'title': 'Accuracy Regresión Logística',
                }}),

        dcc.Graph(id = 'graph2',
            figure={
                'data': [
                    {'x': ['Prueba 1', 'Prueba 2', 'Prueba 3', 'Prueba 4'], 'y': [0.8051470588235294, 0.7871781397792958, 0.8034682080924855, 0.7861271676300579], 'type': 'bar', 'name': 'Accuracy'},
                ],
                'layout': {
                    'title': 'Accuracy Nave - Bayes'
                }

            }),

        dcc.Graph(id = 'graph3',
                figure={
                    'data': [
                        {'x': ['Prueba 1', 'Prueba 2', 'Prueba 3', 'Prueba 4'], 'y': [0.7988445378151261, 0.792433000525486, 0.7961114030478192, 0.7819232790331057], 'type': 'bar', 'name': 'Accuracy'},
                    ],
                    'layout': {
                        'title': 'Accuracy Regresion Logistica + Vectorizador'
                    }})

    ])
    
    ])
    
@app.callback(
    Output('resultado', 'children'),
    Input("dropdown", "value"),
    Input('button', 'n_clicks'),
    Input('button3', 'n_clicks'),
    State('text_area', 'value')
)

def update_output(dropdown_value,n_clicks, n_clicks2, text):
    
    if n_clicks > 0:
        clean_text = limpiar(text)
        if dropdown_value == "bayes":
            x_test=count_vec.transform(clean_text)
            resultado = model_naive.predict(x_test)
            
            
            return '{}'.format(resultado_class(resultado))
       
        elif dropdown_value == "regresionL":
            x_test=count_vec.transform(clean_text)
            resultado = model_logistic.predict(x_test)
            
            return '{}'.format(resultado_class(resultado))
            
        elif dropdown_value == "regresionLV":
            x_test=tfidf_vec.transform(clean_text)
            resultado = model_logistic_2.predict(x_test)
            
            return '{}'.format(resultado_class(resultado))
        
        else:
            x_vec = count_vec.transform(clean_text)
            x_tfidf = tfidf_vec.transform(clean_text)
            
            res_naive = model_naive.predict(x_vec)
            res_LR = model_logistic.predict(x_vec)
            res_LRV = model_logistic_2.predict(x_tfidf)
            
            return 'Usando NAIVE - BAYES: {}'.format(resultado_class(res_naive)) + " \n " + 'Usando Regresión Lineal: {}'.format(resultado_class(res_LR)) + " \n " + 'Usando Regresión Lineal +  Vectorizador: {}'.format(resultado_class(res_LRV)) 

    
@app.callback(Output('button','n_clicks'), #función que limpia los datos 
             [Input('button2','n_clicks')])

def update(reset):
    return 0

@app.callback(Output('graph-container', 'style'), #función que muestra/oculta las gráficas
              [Input('button3','n_clicks')])

def hide_graph(n_clicks2):
    print(n_clicks2)
    if n_clicks2%2 == 1:
        
        return {'display':'block'}
    else:
        return {'display':'none'}
    

@app.callback(Output('model_text', 'children'), #funció
              Input('dropdown','value'),
              Input('button', 'n_clicks'))

def model(value_2, clicks):
    
    if clicks > 0:

        if value_2 == "bayes":

            return '{}'.format("Modelo: Naive - Bayes")

        elif value_2 == "regresionL":

             return '{}'.format("Modelo: Regresión Logstica")

        elif value_2 == "regresionLV":

            return '{}'.format("Modelo: Regresión Logística + Vectorizador")

        else:
            print("hola")        
            return '{}'.format("Modelo: TODOS")
    

@app.callback(Output('resultado', 'style'),
              [Input('resultado','children')])

def hide_graph(text):
    print(text)
    if text == "El Tweet no es de un desastre":
        return {'whiteSpace': 'pre-line', "font-size": "x-large", 'font-family':'Courier New', 'color':'#228B22'}
    elif text == "Es poco probable que el tweet sea de un desastre":
        return {'whiteSpace': 'pre-line', "font-size": "x-large", 'font-family':'Courier New', 'color':'#FFB90F'}
    elif text == "Es bastante probable que el tweet sea de un desastre":
        return {'whiteSpace': 'pre-line', "font-size": "x-large", 'font-family':'Courier New', 'color':'#FF6103'}
    elif text == "Es altamente probable que el tweet sea de un desastre":
        return {'whiteSpace': 'pre-line', "font-size": "x-large", 'font-family':'Courier New', 'color':'#B22222'}
    elif text == "El tweet es de un desastre":
        return {'whiteSpace': 'pre-line', "font-size": "x-large", 'font-family':'Courier New', 'color':'#CD3333'}
    else: 
        return {'whiteSpace': 'pre-line', "font-size": "large", 'font-family':'Courier New'}
    




In [34]:
if __name__ == '__main__': 
    app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [21/Nov/2021 13:45:16] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Nov/2021 13:45:16] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [21/Nov/2021 13:45:16] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [21/Nov/2021 13:45:16] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [21/Nov/2021 13:45:16] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [21/Nov/2021 13:45:16] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [21/Nov/2021 13:45:16] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Nov/2021 13:45:16] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Nov/2021 13:45:16] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Nov/2021 13:45:16] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Nov/2021 13:45:16] "POST /_dash-update-component HTTP/1.

0
None


127.0.0.1 - - [21/Nov/2021 13:45:17] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Nov/2021 13:45:17] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Nov/2021 13:45:17] "POST /_dash-update-component HTTP/1.1" 200 -


1
None


127.0.0.1 - - [21/Nov/2021 13:45:20] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Nov/2021 13:45:20] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Nov/2021 13:45:20] "POST /_dash-update-component HTTP/1.1" 200 -


2
None
